In [15]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import cv2 as cv

In [16]:
DATAPATH = r"C:\Users\yuvfr\proj_university\swedish-leaf-dataset"

In [127]:
def load_image(filename, height=224, width=224):
    img = cv.imread(filename, cv.IMREAD_COLOR)
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (height, width))
    return img

In [77]:
data_dict = dict()
classes = [f'leaf{i}' for i in range(1,16)]

for i, leaf in enumerate(classes):
    
    leaf_path = os.path.join(DATAPATH,leaf)
    print(f"load {leaf}")
    leaf_imgs_list = []
    
    for img_fname in os.listdir(leaf_path):
        leaf_imgs_list.append(load_image(os.path.join(leaf_path, img_fname)))
            
            
    data_dict[leaf] = {'data':np.stack(leaf_imgs_list), 'label':np.repeat(i, len(leaf_imgs_list)).reshape(-1,1)}
    
    print(f"{len(leaf_imgs_list)} images","\n")

load leaf1
(2508, 1423, 3)
(2143, 1147, 3)
(1994, 1294, 3)
(2482, 1457, 3)
(1580, 917, 3)
(1154, 779, 3)
(1304, 791, 3)
(1379, 804, 3)
(2207, 1508, 3)
(1705, 1080, 3)
(1718, 1068, 3)
(2207, 1106, 3)
(1303, 1118, 3)
(1579, 930, 3)
(1793, 1030, 3)
(1379, 867, 3)
(1618, 1005, 3)
(1842, 1206, 3)
(2131, 1206, 3)
(2156, 1194, 3)
(1379, 1018, 3)
(1191, 879, 3)
(2045, 1046, 3)
(2094, 1235, 3)
(1843, 1131, 3)
(1404, 841, 3)
(2031, 1272, 3)
(2018, 1193, 3)
(1655, 1005, 3)
(1530, 880, 3)
(1580, 1017, 3)
(2231, 1306, 3)
(1505, 867, 3)
(1818, 1105, 3)
(1204, 729, 3)
(1104, 753, 3)
(1743, 1219, 3)
(2043, 1457, 3)
(2144, 1419, 3)
(1981, 1244, 3)
(2533, 1420, 3)
(1981, 1035, 3)
(1680, 1118, 3)
(2044, 1193, 3)
(1129, 754, 3)
(1379, 841, 3)
(2093, 1194, 3)
(1216, 791, 3)
(2144, 1269, 3)
(1279, 833, 3)
(1768, 1180, 3)
(1818, 1206, 3)
(1530, 967, 3)
(1204, 734, 3)
(1229, 854, 3)
(1078, 792, 3)
(2257, 1322, 3)
(917, 609, 3)
(1455, 921, 3)
(1706, 1035, 3)
(1431, 1013, 3)
(1592, 1146, 3)
(2120, 1197, 3)
(170

KeyboardInterrupt: 

In [18]:
data = np.vstack([data_dict[k]['data'] for k in data_dict.keys()])
labels = np.repeat(np.arange(1,16), 75)

In [20]:
pretrain_preprocess = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [115]:
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import numpy as np
import json
import requests
import matplotlib.pyplot as plt

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Using {device} for PyTorch')

Using cuda for PyTorch


In [146]:
preprocess = transforms.Compose([
    transforms.ToTensor(),
#     ZeroOneScale(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [182]:
from torch.utils.data import Dataset, DataLoader

class LeafDataset(Dataset):
    def __init__(self, main_dir, loader, transform, label):
        self.main_dir = main_dir
        self.img_list = os.listdir(main_dir)
        self.loader = loader
        self.transform = transform
        self.label = label

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.img_list[idx])
        image = self.loader(img_loc)
        tensor_image = self.transform(image)
        return tensor_image, self.label

In [181]:
def split_leaf_datasets(datasets, test_perc):
    splits = [torch.utils.data.random_split(
        data, lengths=(
            int(len(data)*(1-test_perc)), int(len(data)*test_perc) + 1)) 
              for data in datasets]
    
    train = torch.utils.data.ConcatDataset([split[0] for split in splits])
    valid = torch.utils.data.ConcatDataset([split[1] for split in splits])
    return train, valid

In [227]:
leaf9_ds = LeafDataset(os.path.join(DATAPATH, 'leaf9'), load_image, preprocess, 9)
leaf10_ds = LeafDataset(os.path.join(DATAPATH, 'leaf10'), load_image, preprocess, 10)

In [174]:
train, valid = split_leaf_datasets([leaf9_ds,leaf10_ds], 0.1)

In [183]:
train_loader = DataLoader(train, batch_size=4)
valid_loader = DataLoader(valid, batch_size=4)

In [212]:
class ResNetTransfer(torch.nn.Module):

    def __init__(self):
        super().__init__()
        self.resnet = models.resnet18(pretrained=True).to(device)
        resnet_fc_in_features = self.resnet.fc.in_features
        self.remove_resnet_fc_layer()
        self.resnet.eval()  # to be not optimized and use all neurons w/o dropout/batchnorm etc..
        self.fc = torch.nn.Linear(resnet_fc_in_features, 2).to(device)

    def remove_resnet_fc_layer(self):
        class IdentityLayer(torch.nn.Module):
            def __init__(self):
                super().__init__()

            def forward(self, x):
                return x

        self.resnet.fc = IdentityLayer()


    def forward(self, x):
        with torch.no_grad():
            features = self.resnet(x)
        out = self.fc(x)
        return out

In [213]:
from time import time
import copy


class Trainer:

    def __init__(self, model, criterion, optimizer, num_epochs=25):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.num_epochs = num_epochs

    def make_epoch(self, dataloader, train=True):
        running_num_examples = 0
        running_loss = 0.0
        running_corrects = 0
        with torch.set_grad_enabled(train):
            for inputs, labels in dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = self.model(inputs)
                preds = torch.argmax(outputs, 1)
                loss = self.criterion(outputs, labels)
                if train:
                    loss.backward()
                    self.optimizer.step()
                    self.optimizer.zero_grad()

                # statistics
                running_num_examples += inputs.shape[0]
                running_loss += loss.item() * inputs.shape[0]
                running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / running_num_examples
        epoch_acc = float(running_corrects) / running_num_examples
        self.print_epoch_stats(epoch_loss, epoch_acc)

        return epoch_loss, epoch_acc

    @staticmethod
    def print_epoch_stats(epoch_loss, epoch_acc, train=True):
        print(f'{"train" if train else "valid"} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

    def train(self, train_loader, val_loader):
        start = time()
        best_model_weights = copy.deepcopy(self.model.state_dict())
        best_acc = 0.0

        for epoch in range(self.num_epochs):
            print('Epoch {}/{}'.format(epoch, self.num_epochs - 1))
            print('-' * 10)
            
            self.model.train()
            train_loss, train_acc = self.make_epoch(train_loader, train=True)
            self.model.eval()
            val_loss, val_acc = self.make_epoch(val_loader, train=False)

            # deep copy the model
            if val_acc > best_acc:
                best_acc = val_acc
                best_model_weights = copy.deepcopy(self.model.state_dict())

        time_elapsed = time() - start
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print('Best val Acc: {best_acc:4f}')

        # load best model weights
        self.model.load_state_dict(best_model_weights)

In [214]:
resnet_transfer = ResNetTransfer()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resnet_transfer.fc.parameters(), lr=0.001, momentum=0.9)

trainer = Trainer(resnet_transfer, criterion, optimizer)

In [223]:
class ResNetTransfer(torch.nn.Module):

    def __init__(self, fine_tune=True):
        super().__init__()
        self.resnet = models.resnet18(pretrained=True).to(device)
        resnet_fc_in_features = self.resnet.fc.in_features
        self.remove_resnet_fc_layer()
        self.fc = torch.nn.Linear(resnet_fc_in_features, 2).to(device)
        if not fine_tune:
            self.freeze_base_model()
        self.fine_tune = fine_tune

    def remove_resnet_fc_layer(self):
        class IdentityLayer(torch.nn.Module):
            def __init__(self):
                super().__init__()

            def forward(self, x):
                return x

        self.resnet.fc = IdentityLayer()

    def freeze_base_model(self):
        for param in self.resnet.parameters():
            param.requires_grad = False
        self.resnet.eval()  # to be not optimized and use all neurons w/o dropout/batchnorm etc..

    def forward(self, x):
        with torch.set_grad_enabled(self.fine_tune):
            features = self.resnet(x)
        out = self.fc(features)
        # out = torch.argmax(fc_out, 1).double().view(-1, 1)
        return out

In [251]:
model = ResNetTransfer()
model.load_state_dict(torch.load(os.path.join(DATAPATH, "model.pt")))
model.to('cpu')
model.eval()

ResNetTransfer(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [252]:
test9 = torch.stack([leaf9_ds[i][0] for i in range(10)])
test10 = torch.stack([leaf10_ds[i][0] for i in range(10)])

In [254]:
out = model(test9)
out

tensor([[ 0.3372, -0.7167],
        [-0.0251, -0.7810],
        [ 0.1681, -0.6588],
        [-0.5023, -1.3448],
        [-0.1434, -0.9854],
        [ 0.0537, -1.2923],
        [ 0.0589, -0.6024],
        [-0.1913, -0.4254],
        [-0.0597, -0.9748],
        [-0.1912, -0.8173]], grad_fn=<AddmmBackward0>)

In [255]:
torch.argmax(out,1)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [257]:
out10 = model(test10)
out10

tensor([[-0.1114,  0.8221],
        [-1.0072,  0.8887],
        [-1.0156,  0.9057],
        [-1.6461,  0.0758],
        [-1.7470,  0.3160],
        [-1.8434,  0.5084],
        [-1.3324,  0.7622],
        [-0.4403,  1.4134],
        [-1.0851,  0.5680],
        [-1.3497,  0.6165]], grad_fn=<AddmmBackward0>)

In [259]:
torch.argmax(out10,1)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [221]:
lin = torch.nn.Linear(128,2)

In [222]:
isinstance(lin, torch.nn.Module)

True

In [ ]:
for param in transfer_resnet.parameters():
    param.requires_grad = False